In [ ]:
import numpy as np
from common_function import newmark
import os
import pandas as pd
import re
from matplotlib import pyplot as plt

In [ ]:
w0 = 2 * np.pi / 0.56
k0 = 192 * 1e6
m0 = 1500 * 1e3
xi = 0.05
c0 = xi * 2 * m0 * w0
M0 = np.array(m0, ndmin=2, dtype=float)
C0 = np.array(c0, ndmin=2, dtype=float)
K0 = np.array(k0, ndmin=2, dtype=float)
E0 = np.array(1, ndmin=2, dtype=float)
wave = 0
dt = 0
for root, dirs, files in os.walk("./waves", topdown=False):
    for name in files:
        print(name)
        wave_name = name.split('.')[0]
        if name.endswith(".AT2"):
            filename = os.path.join(root, name)
            with open(filename, 'r') as fo:
                for i in range(3):
                    fo.readline()
                line = fo.readline()
                target = re.search(r'DT= *(.*) *SEC', line, re.I | re.M)
                dt = float(target.group(1))
                wave = pd.read_table(filename, skiprows=4, sep="  ", header=None, engine='python')
                wave = wave.fillna(0)
                wave = wave.values
                size = wave.size
                wave = wave.reshape((size,))
                wave = wave / max(abs(wave)) * 1.4
        if name.endswith(".txt"):
            filename = os.path.join(root, name)
            wave = np.loadtxt(filename)
            wave = wave / max(abs(wave)) * 1.4
            dt = 0.02
resp0 = newmark(K0, M0, C0, wave, dt, E0)

In [ ]:
length = len(wave)

In [ ]:
T = np.arange(0, dt * length, dt)
plt.plot(T, resp0[0][0])

In [ ]:
plt.plot(T, resp0[1][0])

In [ ]:
plt.plot(T, resp0[3][0])

In [ ]:
y = resp0[0][0]
dy = resp0[1][0]
ddy_ab = resp0[3][0]

In [ ]:
data = pd.DataFrame(data=None, columns=["cur_wave", "pre_wave", "str_cur_disp", "str_cur_vel", "str_cur_acc", "str_pre_disp", "str_pre_vel", "str_pre_acc", "str_next_disp", "str_next_vel", "str_next_acc"])

In [ ]:
# 构造样本数据
length = len(wave)
for i in range(1, length - 1):
    row_data = np.array([wave[i], wave[i -1], y[i], dy[i], ddy_ab[i], y[i - 1], dy[i - 1], ddy_ab[i - 1], y[i + 1], dy[i + 1], ddy_ab[i + 1]])
    data.loc[len(data)] = row_data

In [ ]:
x_label = ["cur_wave", "pre_wave", "str_cur_disp", "str_cur_vel", "str_cur_acc", "str_pre_disp", "str_pre_vel", "str_pre_acc"]
y_label = ["str_next_disp", "str_next_vel", "str_next_acc"]

X = data.loc[:, x_label].values
y = data.loc[:, y_label].values

In [ ]:
X_train = X[0: 1200, :]
X_test = X[1200: length - 1, :]
y_train_disp = y[0: 1200, 0]
y_train_vel = y[0: 1200, 1]
y_train_acc = y[0: 1200, 2]
y_test_disp = y[1200: length - 1, 0]
y_test_vel = y[1200: length - 1, 1]
y_test_acc = y[1200: length - 1, 2]

In [ ]:
# 特征标准化，采用最大最小值标准化，转化后的值范围（0,1）
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
new_X_train = X_train
new_X_test = X_test
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
# normalizer = Normalizer(copy=True, norm='l2').fit(new_X_train)
# new_X_train = normalizer.transform(new_X_train)
# new_X_test = normalizer.transform(new_X_test)
ss = StandardScaler().fit(new_X_train)
new_X_train = ss.transform(new_X_train)
new_X_test = ss.transform(new_X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
GBoost_disp = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)
GBoost_vel = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)
GBoost_acc = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
def plot_regression_results(ax, y_true, y_pred, title, scores):
    """预测目标与真实目标的散点图。"""
    ax.plot([y_true.min(), y_true.max()],
            [y_true.min(), y_true.max()],'--r', linewidth=2)
    ax.scatter(y_true, y_pred, alpha=0.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    ax.set_xlim([y_true.min(), y_true.max()])
    ax.set_ylim([y_true.min(), y_true.max()])
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    extra = plt.Rectangle((0, 0), 0, 0, fc="w", fill=False,
                          edgecolor='none', linewidth=0)
    ax.legend([extra], [scores], loc='upper left')
    ax.set_title(title)


def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
GBoost_disp.fit(new_X_train, y_train_disp)
GBoost_vel.fit(new_X_train, y_train_vel)
GBoost_acc.fit(new_X_train, y_train_acc)

train_pred_disp = GBoost_disp.predict(new_X_train)
pred_disp = GBoost_disp.predict(new_X_test)
print(rmsle(y_train_disp, train_pred_disp))

train_pred_vel = GBoost_vel.predict(new_X_train)
pred_vel = GBoost_vel.predict(new_X_test)
print(rmsle(y_train_vel, train_pred_vel))

train_pred_acc = GBoost_acc.predict(new_X_train)
pred_acc = GBoost_acc.predict(new_X_test)
print(rmsle(y_train_acc, train_pred_acc))

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
r2_disp = r2_score(y_test_disp, pred_disp)
mse_disp = mean_squared_error(y_test_disp, pred_disp)
mape_disp = mean_absolute_percentage_error(y_test_disp, pred_disp)

plt.figure(figsize=(10,8))
ax = plt.gca()
plot_regression_results(
        ax, y_test_disp, pred_disp,
        "Stacking Regressor",
        (r'$R^2={:.2f}$' + '\n' + r'$MAPE={:.2f}$' + '\n' + r'$MSE={:.2f}$')
        .format(r2_disp, mse_disp, mape_disp))


r2_vel = r2_score(y_test_vel, pred_vel)
mse_vel = mean_squared_error(y_test_vel, pred_vel)
mape_vel = mean_absolute_percentage_error(y_test_vel, pred_vel)

plt.figure(figsize=(10,8))
ax = plt.gca()
plot_regression_results(
        ax, y_test_vel, pred_vel,
        "Stacking Regressor",
        (r'$R^2={:.2f}$' + '\n' + r'$MAPE={:.2f}$' + '\n' + r'$MSE={:.2f}$')
        .format(r2_vel, mse_vel, mape_vel))


r2_acc = r2_score(y_test_acc, pred_acc)
mse_acc = mean_squared_error(y_test_acc, pred_acc)
mape_acc = mean_absolute_percentage_error(y_test_acc, pred_acc)

plt.figure(figsize=(10,8))
ax = plt.gca()
plot_regression_results(
        ax, y_test_acc, pred_acc,
        "Stacking Regressor",
        (r'$R^2={:.2f}$' + '\n' + r'$MAPE={:.2f}$' + '\n' + r'$MSE={:.2f}$')
        .format(r2_acc, mse_acc, mape_acc))

In [ ]:
# 绘图
T = np.arange(0, dt * length, dt)

In [ ]:
disp = resp0[0][0]
vel = resp0[1][0]
acc = resp0[3][0]
pred_disp = np.zeros(length)
pred_vel = np.zeros(length)
pred_acc = np.zeros(length)

pre_wave = 0
str_pre_disp = 0
str_pre_vel = 0
str_pre_acc = 0

for i in range(0, length - 1):
    cur_wave = wave[i]
    str_cur_disp = disp[i]
    str_cur_vel = vel[i]
    str_cur_acc = acc[i]

    x = np.array([cur_wave, pre_wave, str_cur_disp, str_cur_vel, str_cur_acc, str_pre_disp, str_pre_vel, str_pre_acc], ndmin=2)
    x = ss.transform(x)
    pred_disp[i + 1] = GBoost_disp.predict(x)[0]
    pred_vel[i + 1] = GBoost_vel.predict(x)[0]
    pred_acc[i + 1] = GBoost_acc.predict(x)[0]

    pre_wave = cur_wave
    str_pre_disp = str_cur_disp
    str_pre_vel = str_cur_vel
    str_pre_acc = str_cur_acc



In [ ]:
plt.plot(T, disp, T, pred_disp)

In [ ]:
plt.plot(T, vel, T, pred_vel)

In [ ]:
plt.plot(T, acc, T, pred_acc)